In [ ]:
import scri
import matplotlib.pyplot as plt
from pathlib import Path
from spherical_functions import LM_index as lm
from scipy.interpolate import interp1d
import json
import h5py
import numpy as np
import pandas as pd
import pickle
import itertools
import pickle
import re

plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (12,10)

# Functions

In [ ]:
def moving_average_valid(array,avg_len):
    return np.convolve(array,np.ones(avg_len),'valid')/avg_len

def load_and_pickle(data_path:Path, reload_data:bool=False, data_type:str='abd', options:dict={}):
  if not data_path.exists():
    raise Exception(f"{data_path} does not exist!")

  saved_data_path = data_path.parent/"saved.pkl"
  
  if saved_data_path.exists() and reload_data == False:
    with open(saved_data_path, 'rb') as f:
      saved_data = pickle.load(f)
      print(f"Saved data loaded: {saved_data_path}")
  else:
    saved_data = {}
    if data_type == 'abd':
      saved_data['abd']= scri.create_abd_from_h5(
          file_name=str(data_path),
          file_format="spectrecce_v1",
          **options
        )
      with open(saved_data_path, 'wb') as f:
        pickle.dump(saved_data,f)
      print(f"Data loaded and saved at : {saved_data_path}")

  return saved_data

def load_bondi_constraints(data_path:Path):
  if not data_path.exists():
    raise Exception(f"{data_path} does not exist!")
  saved_data_path = data_path.parent/"saved.pkl"
  if not saved_data_path.exists():
    raise Exception(f"{saved_data_path} does not exist")
  else:
    with open(saved_data_path, 'rb') as f:
      saved_data = pickle.load(f)
      if 'bondi_violation_norms' in saved_data:
        print(f"bondi_violation_norms loaded for {data_path}")
      else:
        print(f"Computing bondi_violation_norms for: {data_path}")
        saved_data['bondi_violation_norms'] = saved_data['abd'].bondi_violation_norms
        with open(saved_data_path, 'wb') as f:
          pickle.dump(saved_data,f)

        print(f"Saved bondi_violation_norms for: {data_path}")
    return saved_data

def add_bondi_constraints(abd_data:dict):
  for key in abd_data:
    abd_data[key]['bondi_violation_norms'] = abd_data[key]["abd"].bondi_violation_norms
    print(f"bondi_violation_norms computed for {key}")

def create_diff_dict_cce(WT_data_dict:dict, l:int, m:int, base_key:str, t_interpolate:np.ndarray):
  h = WT_data_dict[base_key]['abd'].h.interpolate(t_interpolate)
  diff_dict = {"t": h.t}
  y_base = h.data[:,  lm(l,m,h.ell_min)]
  y_norm = np.linalg.norm(y_base)
  for key in WT_data_dict:
    if key == base_key:
      continue
    h = WT_data_dict[key]['abd'].h.interpolate(t_interpolate)
    y_inter = h.data[:,  lm(l,m,h.ell_min)]
    diff_dict[key+"_diff"] = y_inter-y_base
    diff_dict[key+"_absdiff"] = np.abs(y_inter-y_base)
    diff_dict[key+"_rel_diff"] = (y_inter-y_base)/y_norm
    diff_dict[key+"_rel_absdiff"] = np.abs(y_inter-y_base)/y_norm
  return diff_dict

In [ ]:
t_interpolate = np.linspace(-1000,4000,num=5000)
data_path = Path("/groups/sxs/hchaudha/spec_runs/6_segs/6_set1_L3/GW_data_lev3/BondiCceR0100/red_cce.h5")
data = load_and_pickle(data_path,options = {'t_interpolate':t_interpolate})
data = load_bondi_constraints(data_path)

# Multiple ABDs

In [ ]:
cce_data= {}
# cce_data["high_accuracy_Lev0_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev0/BondiCceR0257/red_cce.h5")
# cce_data["high_accuracy_Lev1_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev1/BondiCceR0257/red_cce.h5")
# cce_data["high_accuracy_Lev2_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev2/BondiCceR0257/red_cce.h5")
# cce_data["high_accuracy_Lev3_R0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev3/BondiCceR0258/red_cce.h5")
# cce_data["high_accuracy_Lev4_R0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev4/BondiCceR0258/red_cce.h5")
# cce_data["high_accuracy_Lev5_R0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev5/BondiCceR0258/red_cce.h5")
# cce_data["master_Lev0_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data/BondiCceR0257/red_cce.h5")
# cce_data["master_Lev1_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data_lev1/BondiCceR0257/red_cce.h5")
# cce_data["master_Lev2_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data_lev2/BondiCceR0257/red_cce.h5")
# cce_data["master_Lev3_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_lev3/BondiCceR0257/red_cce.h5")
# cce_data["master_Lev4_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_lev4/BondiCceR0257/red_cce.h5")
# cce_data["master_Lev5_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_lev5/BondiCceR0257/red_cce.h5")
# cce_data["Lev5_bg_ah100_cd_01_uamr_full_R0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_cd_01_uamr_full/GW_data/BondiCceR0258/red_cce.h5")
# cce_data["Lev5_bg_ah100_cd_01_uamr_full_R0686"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_cd_01_uamr_full/GW_data/BondiCceR0686/red_cce.h5")
# cce_data["Lev5_bg_ah100_lapse_uamr_fullR0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_lapse_uamr_full/GW_data/BondiCceR0258/red_cce.h5")
# cce_data["Lev5_bg_ah100_lapse_uamr_full_R0100"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_lapse_uamr_full/GW_data/BondiCceR0100/red_cce.h5")
# cce_data["high_accuracy_Lev5_R0258_ZeroNonSmooth"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/ZeroNonSmooth/red_Lev5_R0258_VolumeData.h5")
# cce_data["high_accuracy_Lev5_R0258_NoIncomingRadiation"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/NoIncomingRadiation/red_Lev5_R0258_VolumeData.h5")
# cce_data["high_accuracy_Lev5_R0258_InverseCubic"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/InverseCubic/red_Lev5_R0258_VolumeData.h5")
# cce_data["high_accuracy_Lev5_R0258_ConformalFactor10"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/ConformalFactor10/red_Lev5_R0258_VolumeData.h5")
# cce_data["high_accuracy_Lev5_R0258_ConformalFactor7"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/ConformalFactor7/red_Lev5_R0258_VolumeData.h5")
# cce_data["high_accuracy_Lev5_R0258_ConformalFactor3"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/ConformalFactor3/red_Lev5_R0258_VolumeData.h5")

# cce_data["Lev01_test_ode_Lev2_257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev2/BondiCceR0257/red_cce.h5")
# cce_data["Lev01_test_ode_Lev1_257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev1/BondiCceR0257/red_cce.h5")

# cce_data["Lev01_test_Lev2_257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data_lev2/BondiCceR0257/red_cce.h5")

levs,run_sets,radius= [],[],[]
# levs = [0,1,2,3,4,5,6]
# levs = [4,5,6]
levs = [0,1,2,3]
# run_sets = [1,2,3]
run_sets = [1]
# radius = [250]
# radius = [100,150,200,250,300,350,500,700,900]
radius = [250,350,700,900]

for l,s,r in itertools.product(levs,run_sets,radius):
  if s == 2 and (l == 0 or l==1):
    continue
  if l <= 3:
    cce_data[f"6_set{s}_L3s{l}_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/6_segs/6_set{s}_L3/GW_data_lev{l}/BondiCceR0{r}/red_cce.h5")
  else:
    cce_data[f"6_set{s}_L6s{l}_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/6_segs/6_set{s}_L6/GW_data_lev{l}/BondiCceR0{r}/red_cce.h5")


fail_flag=False
for key in cce_data:
  if not cce_data[key].exists():
    fail_flag = True
    print(f"{cce_data[key]} does not exist!")
  if fail_flag:
    raise Exception("Some paths do not exist!")

In [ ]:
t_interpolate = np.linspace(-1000,10000,num=5000)
# t_interpolate = np.linspace(-1000,4000,num=5000)

abd_data = {}
for key in cce_data:
  abd_data[key] = load_and_pickle(cce_data[key],options = {'t_interpolate':t_interpolate})
  abd_data[key] = load_bondi_constraints(cce_data[key])

print(abd_data.keys())

In [ ]:
bondi_norms_to_plot = [0,1,2,3,4,5]
bondi_norms_to_plot = [5]

t_min = -10000
t_min = 2000
# t_min = 2500
t_max = 10000
# t_max = 7751.5
# t_max = 7000
t_max = 4000
# t_max = 3500

labels = None
# labels={
# #  "high_accuracy_Lev5_R0258" : "Lev5_ode_tol_R0258",
#  "6_set1_L3s3_200" : "Lev3_variant_R0200",
#  "6_set1_L3s3_250" : "Lev3_variant_R0250",
#  "6_set1_L3s3_300" : "Lev3_variant_R0300",
#  "6_set1_L3s2_200" : "Lev2_variant_R0200",
#  "6_set1_L3s2_250" : "Lev2_variant_R0250",
#  "6_set1_L3s2_300" : "Lev2_variant_R0300",
# }

for key in abd_data:
  violation_dict = abd_data[key]['bondi_violation_norms']
  if "Factor" in key:
    continue
  # if "L3s2" not in key:
  #   continue
  # if "Lev2" not in key:
  #   continue

  t_arr = abd_data[key]["abd"].t
  trimmed_indices = (t_arr>t_min) & (t_arr<t_max)
  t_arr = t_arr[trimmed_indices]

  for i in bondi_norms_to_plot:
    if labels is None:
      plt.semilogy(t_arr,violation_dict[i][trimmed_indices],label=f"{key}_{i}")
    else:
      plt.semilogy(t_arr,violation_dict[i][trimmed_indices],label=f"{labels[key]}_{i}")

  plt.xlabel('t')
  plt.ylabel("bondi violations")
  plt.legend()
  plt.tight_layout()
  # plt.show()
  # plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/CCE/new_set_L3")/f"violations_alllll_{bondi_norms_to_plot}_t={t_min}-{t_max}.png")
  # plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/CCE/")/f"{bondi_norms_to_plot}_t={t_min}-{t_max}.png")

### Plot cce waveform diff

In [ ]:
for i in abd_data.keys():
  print(i)

In [ ]:
modes_to_plot = [(3,2)]
modes_to_plot = [(2,2)]

base_key = 'high_accuracy_Lev5_R0258'
base_key = 'high_accuracy_Lev2_R0257'
# base_key = '6_set1_L6s4_250'
# base_key = '6_set3_L6s6_250'
# base_key = '6_set1_L3s3_250'
base_key = '6_set1_L3s2_250'
radius = int(base_key[-3:])

plot_vals = ["_diff","_absdiff","_rel_diff","_rel_absdiff"]
plot_vals = ["_absdiff","_rel_absdiff"]
# plot_vals = ["_rel_absdiff"]

t_min = 0
# t_min = 2000
# t_min = 2500
# t_min = 1190
t_min = 1210
t_max = 10000
# t_max = 7751.5
# t_max = 7000
t_max = 4000
t_max = 3500
t_interpolate = np.linspace(t_min,t_max,num=5000)

def filter(key):
  if key == base_key:
    return True
  # if '257' not in key:
  #   return True
  # if not re.search(r"Lev[3,4,5]",key):
  #   return True
  if not re.search(r"s[0,1,2]",key):
    return True
  # if 's3' in key:
  #   return True

num_colors_required = len(modes_to_plot)*(len(abd_data)-1)

num_colors_required = 0
for mode,key in itertools.product(modes_to_plot,abd_data):
  if filter(key):
    continue
  num_colors_required = num_colors_required + 1

if num_colors_required <=10:
  cmap = plt.get_cmap('tab10')
  color = [cmap(i) for i in range(num_colors_required )]
elif num_colors_required <=20:
  cmap = plt.get_cmap('tab20')
  color = [cmap(i) for i in range(num_colors_required )]
else:
  color=plt.cm.hsv(np.linspace(0, 1, num_colors_required))
  # color=plt.cm.gist_rainbow(np.linspace(0, 1, num_colors_required))

i=-1
plt.close()
for mode,key in itertools.product(modes_to_plot,abd_data):
  l,m = mode
  if filter(key):
    continue

  diff_dict = create_diff_dict_cce(abd_data,l=l,m=m,base_key=base_key,t_interpolate=t_interpolate)
  t_arr = diff_dict["t"]

  i = i+1
  for plot_val in plot_vals:
    match plot_val:
      case "_rel_absdiff":
        plt.semilogy(t_arr,diff_dict[key+plot_val],label=f"{key}_{mode}"+plot_val,linestyle="-",color=color[i])
      case "_absdiff":
        # plt.semilogy(t_arr,diff_dict[key+plot_val],label=f"{key}_{mode}"+plot_val,linestyle=":",color=color[i])
        plt.semilogy(t_arr,diff_dict[key+plot_val],linestyle=":",color=color[i])
      case "_diff":
        plt.semilogy(t_arr,diff_dict[key+plot_val],label=f"{key}_{mode}"+plot_val,linestyle="--",color=color[i])
      case "_rel_diff":
        plt.semilogy(t_arr,diff_dict[key+plot_val],label=f"{key}_{mode}"+plot_val,linestyle="-.",color=color[i])

    plt.xlabel('t')
    plt.ylabel("diff")
    plt.legend()
    plt.tight_layout()
    plt.title(f"CCE waveform diff {modes_to_plot} {base_key=}")
    # plt.show()
    # plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/CCE/new_set_L3")/f"violations_alllll_{bondi_norms_to_plot}_t={t_min}-{t_max}.png")
    # plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/CCE/")/f"{bondi_norms_to_plot}_t={t_min}-{t_max}.png")


# Finite radius waveform comparison

In [ ]:
def extract_radii(h5_file_path:Path):
  radii = set()
  with h5py.File(h5_file_path,'r') as f:
    names = []
    f.visit(names.append)
  for name in names:
    if "Version" in name:
      continue
    radii.add(name[1:5])
  radii = list(radii)
  radii.sort()
  return radii

In [ ]:
GW_data_path= {}
# GW_data_path["master_Lev0"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data")
# GW_data_path["master_Lev1"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data_lev1")
# GW_data_path["master_Lev2"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data_lev2")
# GW_data_path["master_Lev3"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_lev3")
# GW_data_path["master_Lev4"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_lev4")
# GW_data_path["master_Lev5"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_lev5")

GW_data_path["high_accuracy_Lev0"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev0/rh_FiniteRadii_CodeUnits")
GW_data_path["high_accuracy_Lev1"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev1/rh_FiniteRadii_CodeUnits")
GW_data_path["high_accuracy_Lev2"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev2/rh_FiniteRadii_CodeUnits")
# GW_data_path["high_accuracy_Lev0"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev0_AC/Run/GW2/")
# GW_data_path["high_accuracy_Lev1"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev1_AC/Run/GW2/")
# GW_data_path["high_accuracy_Lev2"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev2_AC/Run/GW2/")
GW_data_path["high_accuracy_Lev3"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev3/rh_FiniteRadii_CodeUnits")
GW_data_path["high_accuracy_Lev4"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev4/rh_FiniteRadii_CodeUnits")
# GW_data_path["high_accuracy_Lev45"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev45_AC/Run/GW2/")
GW_data_path["high_accuracy_Lev5"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev5/rh_FiniteRadii_CodeUnits")
# GW_data_path["high_accuracy_Lev55"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/Lev55_AC/Run/GW2/")


levs,run_sets,radius= [],[],[]
levs = [0,1,2,3,4,5,6]
# levs = [0,1,2,3]
# levs = [4,5,6]
# levs = [0]
run_sets = [3]
# run_sets = [1]

for l,s in itertools.product(levs,run_sets):
  if s == 2 and (l == 0 or l==1):
    continue
  if l <= 3:
    GW_data_path[f"6_set{s}_L3s{l}"] = Path(f"/groups/sxs/hchaudha/spec_runs/6_segs/6_set{s}_L3/GW_data_lev{l}/rh_FiniteRadii_CodeUnits")
  else:
    GW_data_path[f"6_set{s}_L6s{l}"] = Path(f"/groups/sxs/hchaudha/spec_runs/6_segs/6_set{s}_L6/GW_data_lev{l}/rh_FiniteRadii_CodeUnits")

file = "rh_FiniteRadii_CodeUnits.h5"
for key in GW_data_path:
  GW_data_path[key] = GW_data_path[key]/file
  if not GW_data_path[key].exists():
    raise Exception(f"{key}, {GW_data_path[key]} does not exists!")
radii_list = extract_radii(list(GW_data_path.values())[0])
print(radii_list)

In [ ]:
radius = radii_list[-1]
# radius = radii_list[4]
GW_data = {}
for key in GW_data_path:
  GW_data[key] = scri.SpEC.read_from_h5(
      f"{GW_data_path[key]}/R{radius}.dir",
      dataType = scri.h,
      frameType = scri.Inertial,
      r_is_scaled_out = True,
      m_is_scaled_out = True,
      )
  print(f"{key} loaded!")

print(GW_data.keys())

In [ ]:

l, m = 2,2
t_max = 2700
t_interpolate = np.linspace(1200+int(radius),t_max+int(radius),num=5000)

base_key = 'master_Lev5'
# base_key = 'master_Lev2'
base_key = 'high_accuracy_Lev5'
# base_key = 'high_accuracy_Lev2'
# base_key = '6_set3_L6s6'
# base_key = '6_set3_L6s5'
# base_key = '6_set1_L3s3'
# base_key = '6_set2_L3s2'
base_h = GW_data[base_key].interpolate(t_interpolate)

def np_abs_func(x):
  return np.abs(np.real(x))
  # return np.abs(x)

for key in GW_data:
  # if 'L6' not in key:
  #   continue
  if not re.search(r'Lev[3,4,5]',key):
    continue
  if key == base_key:
    continue
  h = GW_data[key].interpolate(t_interpolate)
  shited_time = h.t - int(radius)
  diff = h.data[:, lm(l,m,h.ell_min)] - base_h.data[:, lm(l,m,h.ell_min)]
  diff = np_abs_func(diff)
  diff_normalize = diff/np_abs_func(base_h.data[:, lm(l,m,h.ell_min)])
  plt.plot(shited_time,diff,label=f'{key}_{l=}_{m=}')
  # plt.plot(shited_time,diff_normalize,label=f'{key}_rel_{l=}_{m=}')
  # plt.plot(shited_time,base_h.data[:, lm(l,m,h.ell_min)],label=f'{key}_rel2_{l=}_{m=}')
  # plt.plot(shited_time,np_abs_func(base_h.data[:, lm(l,m,h.ell_min)]),label=f'{key}_rel2_{l=}_{m=}')
  # plt.semilogy(h.t,h.energy_flux(),label=f'{key}')

plt.title(f"diff:{base_key}  {l=},{m=}, {radius=}")
plt.xlabel('t')
plt.ylabel("diff")
plt.yscale('log')
plt.legend()
plt.tight_layout()
plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/del/LevVariations")/f"L05_finite_radius_{base_key}_{l=}_{m=}_{radius}_{t_max=}.png")

# World tube data

In [ ]:
def generate_columns(num_cols:int,beta_type=False):
  if beta_type:
    num_cols = num_cols*2
  L_max = int(np.sqrt((num_cols-1)/2))-1
  # print(L_max,np.sqrt((num_cols-1)/2)-1)
  col_names = ['t(M)']
  for l in range(0,L_max+1):
    for m in range(-l,l+1):
      if beta_type:
        if m==0:
          col_names.append(f"Re({l},{m})")
        elif m < 0:
          continue
        else:
          col_names.append(f"Re({l},{m})")
          col_names.append(f"Im({l},{m})")
      else:
        col_names.append(f"Re({l},{m})")
        col_names.append(f"Im({l},{m})")
  return col_names


def WT_to_pandas(horizon_path:Path):
    assert(horizon_path.exists())
    df_dict = {}
    beta_type_list = ['Beta.dat', 'DuR.dat', 'R.dat', 'W.dat']
    with h5py.File(horizon_path,'r') as hf:
        # Not all horizon files may have AhC
        for key in hf.keys():
            if key == "VersionHist.ver":
              continue 
            if key in beta_type_list:
              df_dict[key] = pd.DataFrame(hf[key], columns=generate_columns(hf[key].shape[1],beta_type=True))
            else:
              df_dict[key] = pd.DataFrame(hf[key], columns=generate_columns(hf[key].shape[1]))


    return df_dict


def create_diff_dict(WT_data_dict:dict, mode:str, variable:str, base_key:str):
  diff_dict = {"t(M)":WT_data_dict[base_key][variable]['t(M)']}
  y_base = WT_data_dict[base_key][variable][mode]
  y_norm = np.linalg.norm(y_base)
  for key in WT_data_dict:
    if key == base_key:
      continue
    y = WT_data_dict[key][variable][mode]
    t = WT_data_dict[key][variable]['t(M)']
    y_interpolator = interp1d(t, y, kind='cubic',fill_value='extrapolate')
    y_inter = y_interpolator(diff_dict['t(M)'])
    diff_dict[key+"_diff"] = y_inter-y_base
    diff_dict[key+"_absdiff"] = np.abs(y_inter-y_base)
    diff_dict[key+"_rel_diff"] = (y_inter-y_base)/y_norm
    diff_dict[key+"_rel_absdiff"] = np.abs(y_inter-y_base)/y_norm
  return diff_dict



In [ ]:
WT_data= {}
WT_data["high_accuracy_Lev0_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev0/BondiCceR0257/BondiCceR0257.h5")
WT_data["high_accuracy_Lev1_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev1/BondiCceR0257/BondiCceR0257.h5")
WT_data["high_accuracy_Lev2_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev2/BondiCceR0257/BondiCceR0257.h5")
WT_data["high_accuracy_Lev3_R0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev3/BondiCceR0258/BondiCceR0258.h5")
WT_data["high_accuracy_Lev4_R0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev4/BondiCceR0258/BondiCceR0258.h5")
WT_data["high_accuracy_Lev5_R0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev5/BondiCceR0258/BondiCceR0258.h5")
# WT_data["master_Lev0_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data/BondiCceR0257/BondiCceR0257.h5")
# WT_data["master_Lev1_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data_lev1/BondiCceR0257/BondiCceR0257.h5")
# WT_data["master_Lev2_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data_lev2/BondiCceR0257/BondiCceR0257.h5")
# WT_data["master_Lev3_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_lev3/BondiCceR0257/BondiCceR0257.h5")
# WT_data["master_Lev4_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_lev4/BondiCceR0257/BondiCceR0257.h5")
# WT_data["master_Lev5_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_lev5/BondiCceR0257/BondiCceR0257.h5")
# WT_data["Lev5_bg_ah100_cd_01_uamr_full_R0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_cd_01_uamr_full/GW_data/BondiCceR0258/BondiCceR0258.h5")
# WT_data["Lev5_bg_ah100_cd_01_uamr_full_R0686"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_cd_01_uamr_full/GW_data/BondiCceR0686/BondiCceR0686.h5")
# WT_data["Lev5_bg_ah100_lapse_uamr_fullR0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_lapse_uamr_full/GW_data/BondiCceR0258/BondiCceR0258.h5")
# WT_data["Lev5_bg_ah100_lapse_uamr_full_R0100"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_lapse_uamr_full/GW_data/BondiCceR0100/BondiCceR0100.h5")
# WT_data["high_accuracy_Lev5_R0258_ZeroNonSmooth"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/ZeroNonSmooth/red_Lev5_R0258_VolumeData.h5")
# WT_data["high_accuracy_Lev5_R0258_NoIncomingRadiation"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/NoIncomingRadiation/red_Lev5_R0258_VolumeData.h5")
# WT_data["high_accuracy_Lev5_R0258_InverseCubic"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/InverseCubic/red_Lev5_R0258_VolumeData.h5")
# WT_data["high_accuracy_Lev5_R0258_ConformalFactor10"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/ConformalFactor10/red_Lev5_R0258_VolumeData.h5")
# WT_data["high_accuracy_Lev5_R0258_ConformalFactor7"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/ConformalFactor7/red_Lev5_R0258_VolumeData.h5")
# WT_data["high_accuracy_Lev5_R0258_ConformalFactor3"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/ConformalFactor3/red_Lev5_R0258_VolumeData.h5")

# WT_data["Lev01_test_ode_Lev2_257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev2/BondiCceR0257/BondiCceR0257.h5")
# WT_data["Lev01_test_ode_Lev1_257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev1/BondiCceR0257/BondiCceR0257.h5")

# WT_data["Lev01_test_Lev2_257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data_lev2/BondiCceR0257/BondiCceR0257.h5")

levs,run_sets,radius= [],[],[]
levs = [0,1,2,3,4,5,6]
# levs = [4,5,6]
# levs = [0,1,2,3]
# levs = [0]
# run_sets = [1,2,3]
# run_sets = [1,3]
run_sets = [1]
radius = [250]
# radius = [100,150,200,250,300,350,500,700,900]
# radius = [100,250,900]

for l,s,r in itertools.product(levs,run_sets,radius):
  if s == 2 and (l == 0 or l==1):
    continue
  if l <= 3:
    WT_data[f"6_set{s}_L3s{l}_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/6_segs/6_set{s}_L3/GW_data_lev{l}/BondiCceR0{r}/BondiCceR0{r}.h5")
  else:
    WT_data[f"6_set{s}_L6s{l}_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/6_segs/6_set{s}_L6/GW_data_lev{l}/BondiCceR0{r}/BondiCceR0{r}.h5")


fail_flag=False
for key in WT_data:
  if not WT_data[key].exists():
    fail_flag = True
    print(f"{WT_data[key]} does not exist!")
  if fail_flag:
    raise Exception("Some paths do not exist!")

In [ ]:
WT_data_dict = {}
for key in WT_data:
  WT_data_dict[key] = WT_to_pandas(WT_data[key])
  print(f"{key} loaded!")

print(WT_data_dict.keys())

In [ ]:
variables_to_plot = ['Beta.dat', 'DrJ.dat', 'DuR.dat', 'H.dat', 'J.dat', 'Q.dat', 'R.dat', 'U.dat', 'W.dat']
variables_to_plot = ['DrJ.dat', 'J.dat', 'Q.dat', 'R.dat']
variables_to_plot = ['R.dat']

modes_to_plot = ['Re(2,2)','Im(2,2)','Re(2,0)']
modes_to_plot = ['Re(2,2)']

base_key = 'high_accuracy_Lev5_R0258'
# base_key = '6_set1_L6s6_250'
base_key = '6_set1_L6s6_250'
# base_key = '6_set3_L3s3_250'
# base_key = '6_set1_L3s2_250'
radius = int(base_key[-3:])

plot_vals = ["_diff","_absdiff","_rel_diff","_rel_absdiff"]
plot_vals = ["_absdiff","_rel_absdiff"]
# plot_vals = ["_rel_absdiff"]

t_min = -10000
# t_min = 2000
t_min = 2500
t_min = 1190
t_min = 1210
t_max = 10000
# t_max = 7751.5
# t_max = 7000
t_max = 4000
# t_max = 3500



num_colors_required = len(list(itertools.product(variables_to_plot,modes_to_plot)))*(len(WT_data_dict)-1)

num_colors_required = 0
for variable,mode,key in itertools.product(variables_to_plot,modes_to_plot,WT_data_dict):
  if key == base_key:
    continue
  if '250' not in key:
    continue
  num_colors_required = num_colors_required + 1

if num_colors_required <=10:
  cmap = plt.get_cmap('tab10')
  color = [cmap(i) for i in range(num_colors_required )]
elif num_colors_required <=20:
  cmap = plt.get_cmap('tab20')
  color = [cmap(i) for i in range(num_colors_required )]
else:
  color=plt.cm.hsv(np.linspace(0, 1, num_colors_required))
  # color=plt.cm.gist_rainbow(np.linspace(0, 1, num_colors_required))

i=-1
plt.close()
for variable,mode,key in itertools.product(variables_to_plot,modes_to_plot,WT_data_dict):
  if key == base_key:
    continue
  if '250' not in key:
    continue
  # if not re.search(r"Lev[3,4,5]",key):
  #   continue

  diff_dict = create_diff_dict(WT_data_dict,variable=variable,mode=mode,base_key=base_key)
  t_arr = diff_dict["t(M)"]
  trimmed_indices = (t_arr>t_min) & (t_arr<t_max)
  t_arr = t_arr[trimmed_indices]

  i = i+1
  for plot_val in plot_vals:
    match plot_val:
      case "_rel_absdiff":
        plt.semilogy(t_arr,diff_dict[key+plot_val][trimmed_indices],label=f"{key}_{variable}_{mode}"+plot_val,linestyle="-",color=color[i])
      case "_absdiff":
        # plt.semilogy(t_arr,diff_dict[key+plot_val][trimmed_indices],label=f"{key}_{variable}_{mode}"+plot_val,linestyle=":",color=color[i])
        plt.semilogy(t_arr,diff_dict[key+plot_val][trimmed_indices],linestyle=":",color=color[i])
      case "_diff":
        plt.semilogy(t_arr,diff_dict[key+plot_val][trimmed_indices],label=f"{key}_{variable}_{mode}"+plot_val,linestyle="--",color=color[i])
      case "_rel_diff":
        plt.semilogy(t_arr,diff_dict[key+plot_val][trimmed_indices],label=f"{key}_{variable}_{mode}"+plot_val,linestyle="-.",color=color[i])
    

    plt.xlabel('t')
    plt.ylabel("diff")
    plt.legend()
    plt.tight_layout()
    plt.title(f"Diff {base_key}")
    # plt.show()
    # plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/CCE/new_set_L3")/f"violations_alllll_{bondi_norms_to_plot}_t={t_min}-{t_max}.png")
    # plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/CCE/")/f"{bondi_norms_to_plot}_t={t_min}-{t_max}.png")


### No base diff

In [ ]:
variables_to_plot = ['Beta.dat', 'DrJ.dat', 'DuR.dat', 'H.dat', 'J.dat', 'Q.dat', 'R.dat', 'U.dat', 'W.dat']
# variables_to_plot = ['Beta.dat']

l,m = 2,2
modes_to_plot = ['Re(2,2)','Im(2,2)','Re(2,0)']
modes_to_plot = ['Re(2,2)']

t_min = -10000
# t_min = 2000
t_min = 2500
t_min = 1200
t_max = 10000
# t_max = 7751.5
# t_max = 7000
t_max = 4000
t_max = 3500

labels = None
# labels={
# #  "high_accuracy_Lev5_R0258" : "Lev5_ode_tol_R0258",
#  "6_set1_L3s3_200" : "Lev3_variant_R0200",
#  "6_set1_L3s3_250" : "Lev3_variant_R0250",
#  "6_set1_L3s3_300" : "Lev3_variant_R0300",
#  "6_set1_L3s2_200" : "Lev2_variant_R0200",
#  "6_set1_L3s2_250" : "Lev2_variant_R0250",
#  "6_set1_L3s2_300" : "Lev2_variant_R0300",
# }

for key in WT_data_dict:
  data_dict = WT_data_dict[key]
  if "s3" not in key:
    continue

  t_arr = data_dict[variables_to_plot[0]]["t(M)"]
  trimmed_indices = (t_arr>t_min) & (t_arr<t_max)
  t_arr = t_arr[trimmed_indices]

  for i,mode in itertools.product(variables_to_plot, modes_to_plot):
    if labels is None:
      plt.plot(t_arr,data_dict[i][mode][trimmed_indices],label=f"{key}_{i}_{mode}")
    else:
      plt.plot(t_arr,data_dict[i][mode][trimmed_indices],label=f"{labels[key]}_{i}_{mode}")

  plt.xlabel('t')
  plt.legend()
  plt.tight_layout()
  # plt.show()
  # plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/CCE/new_set_L3")/f"violations_alllll_{bondi_norms_to_plot}_t={t_min}-{t_max}.png")
  # plt.savefig(Path("/home/hchaudha/notes/spec_accuracy/CCE/")/f"{bondi_norms_to_plot}_t={t_min}-{t_max}.png")


# WT data modes plots

## modes vs time plots

In [ ]:
def filter_by_regex(regex,col_list,exclude=False):
  filtered_set = set()
  if type(regex) is list:
    for reg in regex:
      for i in col_list:
        if re.search(reg,i):
          filtered_set.add(i)
  else:
    for i in col_list:
      if re.search(regex,i):
        filtered_set.add(i)

  filtered_list = list(filtered_set)
  if exclude:
    col_list_copy = list(col_list.copy())
    for i in filtered_list:
      if i in col_list_copy:
        col_list_copy.remove(i)
    filtered_list = col_list_copy

  # Restore the original order
  filtered_original_ordered_list = []
  for i in list(col_list):
    if i in filtered_list:
      filtered_original_ordered_list.append(i)
  return filtered_original_ordered_list

def limit_by_col_val(min_val,max_val,col_name,df):
  filter = (df[col_name]>=min_val) &(df[col_name] <=max_val)
  return df[filter]

def abs_mean_value_upto_l(pd_series,L_max:int):
  idx = pd_series.index
  abs_cum_sum = 0
  num = 0
  for i in idx:
    L = int(i.split(",")[0][3:])
    if L > L_max:
      continue
    else:
      abs_cum_sum = abs_cum_sum+abs(pd_series[i])
      num = num +1
  return abs_cum_sum/num


In [ ]:
WT_data= {}
# WT_data["high_accuracy_Lev0_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev0/BondiCceR0257/BondiCceR0257.h5")
# WT_data["high_accuracy_Lev1_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev1/BondiCceR0257/BondiCceR0257.h5")
# WT_data["high_accuracy_Lev2_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev2/BondiCceR0257/BondiCceR0257.h5")
# WT_data["high_accuracy_Lev3_R0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev3/BondiCceR0258/BondiCceR0258.h5")
# WT_data["high_accuracy_Lev4_R0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev4/BondiCceR0258/BondiCceR0258.h5")
# WT_data["high_accuracy_Lev5_R0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/Ev/GW_data_lev5/BondiCceR0258/BondiCceR0258.h5")
# WT_data["master_Lev0_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data/BondiCceR0257/BondiCceR0257.h5")
# WT_data["master_Lev1_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data_lev1/BondiCceR0257/BondiCceR0257.h5")
# WT_data["master_Lev2_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data_lev2/BondiCceR0257/BondiCceR0257.h5")
# WT_data["master_Lev3_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_lev3/BondiCceR0257/BondiCceR0257.h5")
# WT_data["master_Lev4_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_lev4/BondiCceR0257/BondiCceR0257.h5")
# WT_data["master_Lev5_R0257"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/Ev/GW_data_lev5/BondiCceR0257/BondiCceR0257.h5")
# WT_data["Lev5_bg_ah100_cd_01_uamr_full_R0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_cd_01_uamr_full/GW_data/BondiCceR0258/BondiCceR0258.h5")
# WT_data["Lev5_bg_ah100_cd_01_uamr_full_R0686"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_cd_01_uamr_full/GW_data/BondiCceR0686/BondiCceR0686.h5")
# WT_data["Lev5_bg_ah100_lapse_uamr_fullR0258"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_lapse_uamr_full/GW_data/BondiCceR0258/BondiCceR0258.h5")
# WT_data["Lev5_bg_ah100_lapse_uamr_full_R0100"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_lapse_uamr_full/GW_data/BondiCceR0100/BondiCceR0100.h5")
# WT_data["high_accuracy_Lev5_R0258_ZeroNonSmooth"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/ZeroNonSmooth/red_Lev5_R0258_VolumeData.h5")
# WT_data["high_accuracy_Lev5_R0258_NoIncomingRadiation"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/NoIncomingRadiation/red_Lev5_R0258_VolumeData.h5")
# WT_data["high_accuracy_Lev5_R0258_InverseCubic"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/InverseCubic/red_Lev5_R0258_VolumeData.h5")
# WT_data["high_accuracy_Lev5_R0258_ConformalFactor10"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/ConformalFactor10/red_Lev5_R0258_VolumeData.h5")
# WT_data["high_accuracy_Lev5_R0258_ConformalFactor7"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/ConformalFactor7/red_Lev5_R0258_VolumeData.h5")
# WT_data["high_accuracy_Lev5_R0258_ConformalFactor3"] = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations/initial_data/ConformalFactor3/red_Lev5_R0258_VolumeData.h5")

# WT_data["Lev01_test_ode_Lev2_257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev2/BondiCceR0257/BondiCceR0257.h5")
# WT_data["Lev01_test_ode_Lev1_257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/new_ode_tol/high_accuracy_L35/Ev/GW_data_lev1/BondiCceR0257/BondiCceR0257.h5")

# WT_data["Lev01_test_Lev2_257"] = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev/GW_data_lev2/BondiCceR0257/BondiCceR0257.h5")

levs,run_sets,radius= [],[],[]
levs = [0,1,2,3]
# levs = [4,5,6]
# levs = [0,1,2,3]
levs = [6]
# run_sets = [1,2,3]
# run_sets = [1,3]
run_sets = [1]
# radius = [250]
radius = [100,150,200,250,300,350,500,700,900]
# radius = [100,250,900]

for l,s,r in itertools.product(levs,run_sets,radius):
  if s == 2 and (l == 0 or l==1):
    continue
  if l <= 3:
    WT_data[f"6_set{s}_L3s{l}_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/6_segs/6_set{s}_L3/GW_data_lev{l}/BondiCceR0{r}/BondiCceR0{r}.h5")
  else:
    WT_data[f"6_set{s}_L6s{l}_{r}"] = Path(f"/groups/sxs/hchaudha/spec_runs/6_segs/6_set{s}_L6/GW_data_lev{l}/BondiCceR0{r}/BondiCceR0{r}.h5")


fail_flag=False
for key in WT_data:
  if not WT_data[key].exists():
    fail_flag = True
    print(f"{WT_data[key]} does not exist!")
  if fail_flag:
    raise Exception("Some paths do not exist!")

In [ ]:
WT_data_dict = {}
for key in WT_data:
  WT_data_dict[key] = WT_to_pandas(WT_data[key])
  print(f"{key} loaded!")

print(WT_data_dict.keys())

In [ ]:
data1 = WT_data_dict['6_set1_L3s0_250']['Beta.dat']
data2 = WT_data_dict['6_set1_L3s3_250']['Beta.dat']
diff = data2 - data1
diff['t(M)'] = data2['t(M)']

In [ ]:
diff = WT_data_dict['6_set1_L3s0_250']['W.dat']

In [ ]:
minT = 0
minT = 1200

maxT = 40000
maxT = 4000
data1 = limit_by_col_val(minT,maxT,'t(M)',data1)
data = limit_by_col_val(minT,maxT,'t(M)',diff)

min_clip_val = 1e-25

domain_col_list = filter_by_regex(regex=["Re"],col_list=data.columns)
domain_col_list = filter_by_regex(regex=[r"Re\(\d6"],col_list=domain_col_list)

visual_data = data[domain_col_list]
visual_data = np.abs(visual_data)
min_val = visual_data.min().min()
print(f"Minimum value in the filtered data: {min_val}")
if min_val < min_clip_val:
  print(f"Min value is too small {min_val}, clipping at {min_clip_val}")
  small_number_mask = visual_data < min_clip_val
  visual_data[small_number_mask] = min_clip_val
visual_data = np.log10(visual_data)

column_names = list(visual_data.columns)

vmin_log,vmax_log = -11,-1
vmin_log,vmax_log = None,None
if vmin_log is None:
  vmin_log = visual_data.min().min()
if vmax_log is None:
  vmax_log = visual_data.max().max()

print(vmin_log,vmax_log)

plt.figure(figsize=(45, 10))
imshow_plot = plt.imshow(
    visual_data[column_names], 
    aspect='auto', 
    # cmap='RdYlGn_r', 
    cmap='viridis', 
    origin='lower',
    vmin=vmin_log, 
    vmax=vmax_log
)

plt.xticks(
    ticks=np.arange(len(visual_data.columns)), 
    labels=[i.split(" ")[-1] for i in column_names], 
    rotation=90
)

ytick_step = len(data) // 10  # Show about 10 ticks
plt.yticks(
    ticks=np.arange(0, len(data), ytick_step), 
    labels=data['t(M)'][::ytick_step].astype(int)
)

colorbar = plt.colorbar(imshow_plot)

plt.ylabel('t(M)')
plt.title(f'{key}')
plt.tight_layout()
plt.grid(False)
plt.show()

## Mean/Max of modes over time

In [ ]:
WT_data_dict.keys()

In [ ]:
reduced_dict = {}
L_max_for_mean = 50
variables = ['Beta.dat', 'DrJ.dat', 'DuR.dat', 'H.dat', 'J.dat', 'Q.dat', 'R.dat', 'U.dat', 'W.dat']
variables = ['H.dat', 'R.dat']
variables = ['H.dat']
r1 = '100'
# run_keys = [f'6_set1_L6s6_{r1}']
# run_keys = [f'6_set1_L6s5_{r1}']
# run_keys = [f'6_set1_L6s4_{r1}',f'6_set1_L6s5_{r1}',f'6_set1_L6s6_{r1}']
run_keys = [f'6_set1_L3s0_{r1}',f'6_set1_L3s1_{r1}',f'6_set1_L3s2_{r1}',f'6_set1_L3s3_{r1}']
r1='250'
run_keys = run_keys+[f'6_set1_L3s0_{r1}',f'6_set1_L3s1_{r1}',f'6_set1_L3s2_{r1}',f'6_set1_L3s3_{r1}']
r1='900'
run_keys = run_keys+[f'6_set1_L3s0_{r1}',f'6_set1_L3s1_{r1}',f'6_set1_L3s2_{r1}',f'6_set1_L3s3_{r1}']
run_keys = WT_data_dict.keys()
# run_keys = ['high_accuracy_Lev0_R0257', 'high_accuracy_Lev1_R0257', 'high_accuracy_Lev2_R0257',]
# run_keys = ['high_accuracy_Lev3_R0258', 'high_accuracy_Lev4_R0258', 'high_accuracy_Lev5_R0258',]

minT = 0
minT = 3200

maxT = 40000
maxT = 4000

reduced_dict = {}
flattened_dict = {}
for run_key in run_keys:

  column_names = None
  reduced_dict[run_key] = {}

  for var in variables:

    data = limit_by_col_val(minT,maxT,'t(M)',WT_data_dict[run_key][var])

    domain_col_list = filter_by_regex(regex=["Re"],col_list=data.columns)
    domain_col_list = filter_by_regex(regex=[r"Re\(\d\d?,\d\d?\)"],col_list=domain_col_list)
    # domain_col_list = filter_by_regex(regex=[r"Re\(\d\d,\d\d?\)"],col_list=domain_col_list)

    if column_names is None and var == "R.dat":
      column_names = domain_col_list

    visual_data = data[domain_col_list]
    visual_data = np.abs(visual_data)
    min_val = visual_data.min().min()
    # print(f"Minimum value in the filtered data: {min_val}")

    reduced_dict[run_key][var] = {
      "mean" : visual_data.mean(),
      "rel_mean" : visual_data.mean()/abs_mean_value_upto_l( visual_data.mean(),L_max_for_mean),
      # "run_mean" : visual_data.mean().cumsum()/range(1,1+len( visual_data.mean())),
      "max" : visual_data.max(),
      "rel_max" : visual_data.max()/abs_mean_value_upto_l( visual_data.max(),L_max_for_mean),
      # "run_max" : visual_data.max().cumsum()/range(1,1+len( visual_data.max())),
    }


  for key,val in reduced_dict[run_key].items():
    for key1,val1 in val.items():
      flattened_dict[f"{key[:-4]}_{key1}_{run_key}"] = val1

sorted_keys = sorted(flattened_dict.keys())
flattened_dict = {key: flattened_dict[key] for key in sorted_keys}

df = None
for key,val in flattened_dict.items():
  if df is None:
    df = pd.DataFrame([val],index=[key])
  else:
    df = pd.concat([df,pd.DataFrame([val],index=[key])])

In [ ]:
dfT = df.T
for col in dfT.columns:
  if "max" not in col:
    continue
  if "rel_" in col:
    continue
  plt.plot(dfT[col],label = col)

plt.title(f'{run_key} t(M)={minT},{maxT}')
plt.tight_layout()
plt.legend()
plt.grid(False)
plt.yscale('log')
plt.xticks([])
plt.show()

In [ ]:
cols = dfT.columns
cols_new = [i for i in cols if "run_max" in i]
a = np.arange(1,len(cols_new)+1)
one_over_x = 2.8e-7/a

plt.plot(dfT[cols_new].loc['Re(16,16)'])
plt.plot(one_over_x)

In [ ]:
def get_mode(name):
  return int(name.split("(")[-1].split(")")[0])
def get_radii(name):
  return int(name.split('_')[-1])
def sort_by_power_modes(col_names):
  col_name_copy = list(col_names).copy()
  return sorted(col_name_copy, key=lambda x: int(get_mode(x)))

def add_L_mode_power(df:pd.DataFrame,L:int, ReOrIm:str):
  column_names = df.columns

  n = 0
  power = 0
  for m in range(-L,L+1):
    col_name = f'{ReOrIm}({L},{m})'
    # print(col_name)
    if col_name in column_names:
      power = power + df[col_name]*df[col_name]
      n = n + 1
  if n != 0:
    power = power/n
    df[f'pow_{ReOrIm}({L})'] = power
  return power

def add_all_L_mode_power(df:pd.DataFrame,L_max:int):
  local_df = df.copy()
  total_power_Re = 0
  total_power_Im = 0
  for l in range(0,L_max+1):
    total_power_Re = total_power_Re + add_L_mode_power(local_df,l,"Re")
    total_power_Im = total_power_Im + add_L_mode_power(local_df,l,"Im")
    local_df[f"pow_cum_Re({l})"] = total_power_Re
    local_df[f"pow_cum_Im({l})"] = total_power_Im
  return local_df

In [ ]:
reduced_dict = {}
L_max_for_mean = 50
variables = ['Beta.dat', 'DrJ.dat', 'DuR.dat', 'H.dat', 'J.dat', 'Q.dat', 'R.dat', 'U.dat', 'W.dat']
variables = ['H.dat', 'R.dat']
variables = ['Beta.dat']
# r1 = '100'
# run_keys = [f'6_set1_L6s6_{r1}']
# run_keys = [f'6_set1_L6s5_{r1}']
# run_keys = [f'6_set1_L6s4_{r1}',f'6_set1_L6s5_{r1}',f'6_set1_L6s6_{r1}']
# run_keys = [f'6_set1_L3s0_{r1}',f'6_set1_L3s1_{r1}',f'6_set1_L3s2_{r1}',f'6_set1_L3s3_{r1}']
# r1='250'
# run_keys = run_keys+[f'6_set1_L3s0_{r1}',f'6_set1_L3s1_{r1}',f'6_set1_L3s2_{r1}',f'6_set1_L3s3_{r1}']
# r1='900'
# run_keys = run_keys+[f'6_set1_L3s0_{r1}',f'6_set1_L3s1_{r1}',f'6_set1_L3s2_{r1}',f'6_set1_L3s3_{r1}']
run_keys = WT_data_dict.keys()
# run_keys = ['high_accuracy_Lev0_R0257', 'high_accuracy_Lev1_R0257', 'high_accuracy_Lev2_R0257',]
# run_keys = ['high_accuracy_Lev3_R0258', 'high_accuracy_Lev4_R0258', 'high_accuracy_Lev5_R0258',]

minT = 0
minT = 3200

maxT = 40000
maxT = 4000

power_dict = {}
flattened_dict = {}
for run_key in run_keys:

  column_names = None
  power_dict[run_key] = {}

  for var in variables:

    data = limit_by_col_val(minT,maxT,'t(M)',WT_data_dict[run_key][var])

    power_dict[run_key][var] = add_all_L_mode_power(data,16)
  
  for key,val in power_dict[run_key].items():
    for key1,val1 in val.items():
      if "pow" in key1:
        flattened_dict[f"{key[:-4]}_{key1}_{run_key}"] = val1

sorted_keys = sort_by_power_modes(flattened_dict.keys())
flattened_dict = {key: flattened_dict[key] for key in sorted_keys}
flattened_dict['t(M)'] = val['t(M)']


In [ ]:

minY,maxY = 1e-40,1e-10
avg_len = 1
# avg_len = 1000

last_val_list = []
r_list = [100,150,200,250,300,350,500,700,900]
r_list = [250]
L_list =  [0,1,2,3,4,5,6]
L_list =  list(range(0,17))

for col in flattened_dict:
  if 'cum_' in col:
    continue
  if 'Re' not in col:
    continue
  # if 't(M)' == col:
  #   continue
  L = get_mode(col)
  r = get_radii(col)
  # if "pow_Re" not in col:
  #   continue
  if (r not in r_list) or (L not in L_list):
    continue

  x = flattened_dict['t(M)'][avg_len-1:]
  y = moving_average_valid(flattened_dict[col],avg_len=avg_len)
  last_val_list.append(y[-1])
  plt.plot(x,y,label = col)

plt.title(f'Power in modes, var={variables}, avg_len={avg_len}')
plt.tight_layout()
plt.legend()
plt.grid(False)
plt.yscale('log')
plt.ylabel(f"Power in modes")
# plt.ylim((minY,maxY))
plt.xlabel("t(M)")
plt.show()

In [ ]:
plt.scatter(L_list, last_val_list)
plt.yscale('log')

In [ ]:
from scipy.optimize import curve_fit

one_over_x = last_val_list[0]*1e2/np.array(r_list)
one_over_x2 = last_val_list[0]*1e4/np.array(r_list)**2

# Define the fitting function
def fitting_func(r, a, b):
    return a / r**b
popt, pcov = curve_fit(fitting_func, r_list, last_val_list,p0=[last_val_list[0],2])
a_fit, b_fit = popt

# Create a smooth curve for the fitted function
fit_vals = fitting_func(r_list, a_fit, b_fit)

plt.plot(r_list,last_val_list, label=f"power in mode L={L_list}",marker='x')
# plt.plot(r_list,one_over_x, label="1/x",marker='x')
# plt.plot(r_list,one_over_x2, label="1/(x*x)",marker='x')
plt.plot(r_list,fit_vals,label=f'Fit: a/r^b, a={a_fit:.2e}, b={b_fit:.2f}', linestyle='--')
plt.yscale('log')
plt.legend()
plt.xlabel('r(M)')
plt.ylabel('power in modes')
plt.title(f"L={L_list}")

In [ ]:
flattened_dict['Beta_pow_cum_1_Re_6_set1_L6s6_900']

In [ ]:
dflog10 = df.copy()

# Some values are zero need to deal with those
# min_clip_val = 1e-25
# if min_val < min_clip_val:
#   print(f"Min value in {var} is too small {min_val}, clipping at {min_clip_val}")
#   small_number_mask = visual_data < min_clip_val
#   visual_data[small_number_mask] = min_clip_val
    # visual_data = np.log10(visual_data)

vmin_log,vmax_log = -11,-1
vmin_log,vmax_log = None,None
if vmin_log is None:
  vmin_log = dflog10.min().min()
if vmax_log is None:
  vmax_log = dflog10.max().max()

print(vmin_log,vmax_log)

plt.figure(figsize=(45, 10))
imshow_plot = plt.imshow(
    dflog10[column_names], 
    aspect='auto', 
    cmap='RdYlGn_r', 
    # cmap='viridis', 
    origin='lower',
    vmin=vmin_log, 
    vmax=vmax_log
)

plt.xticks(
    ticks=np.arange(len(dflog10.columns)), 
    labels=[i.split(" ")[-1] for i in column_names], 
    rotation=90
)


plt.yticks(
    ticks=np.arange(0, len(dflog10.index)), 
    labels=dflog10.index
)

colorbar = plt.colorbar(imshow_plot)

# plt.ylabel('t(M)')
plt.title(f'{run_key} t(M)={minT},{maxT}')
plt.tight_layout()
plt.grid(False)
plt.show()

# Rough

In [ ]:
cce_data = Path("/groups/sxs/hchaudha/spec_runs/CCE_stuff/CceExecutables/CharacteristicExtractReduction.h5")
cce_data = Path("/groups/sxs/hchaudha/spec_runs/CCE_stuff/tests/CharacteristicExtractReduction.h5")
cce_data = Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/cce_bondi/Lev0_R0257/red_Lev0_R0257_VolumeData.h5")
# cce_data = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev3_R0258/red_Lev3_R0258_VolumeData.h5")
# cce_data = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev4_R0258/red_Lev4_R0258_VolumeData.h5")
# cce_data = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_R0258/red_Lev5_R0258_VolumeData.h5")
# cce_data = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_master/cce_bondi/Lev5_R0257/red_Lev5_R0257_VolumeData.h5")
# cce_data = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_cd_01_uamr_full/GW_data/BondiCceR0258/red_cce.h5")
# cce_data = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_cd_01_uamr_full/GW_data/BondiCceR0686/red_cce.h5")
# cce_data = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_lapse_uamr_full/GW_data/BondiCceR0258/red_cce.h5")
# cce_data = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_bg_ah100_lapse_uamr_full/GW_data/BondiCceR0100/red_cce.h5")

In [ ]:

abd = scri.create_abd_from_h5(
      file_name=str(cce_data),
      file_format="spectrecce_v1",
      # ch_mass=1.0,  # Optional; helpful if known
      t_interpolate=np.linspace(-1000,10000,num=5000),  # Optional; for some specified values of `t_worldtube`
      # t_0_superrest=3000,
      # padding_time=400
    )


In [ ]:
h = abd.h
l,m=2,2
# l,m=1,1
print(lm(l,m,h.ell_min))
time_cut = 34500
plt.plot(h.t[h.t<time_cut], h.data[:,  lm(l,m,h.ell_min)][h.t<time_cut])

plt.show()

In [ ]:
violations2 = abd.bianchi_2()
diff_abs = (violations2[1] - violations2[0]).norm()
diff_rel = diff_abs/(violations2[0].norm())
h.data.shape,violations2[0].shape,diff_abs.shape,diff_rel.shape

In [ ]:
plt.semilogy(abd.t,diff_abs,label=f"diff_abs")
plt.semilogy(abd.t,diff_rel,label=f"diff_rel")
plt.ylabel("bianchi2")
plt.xlabel("t")
plt.legend()
plt.tight_layout()

In [ ]:
save_folder = cce_data.parent/"violations"
save_folder.mkdir(exist_ok=True)
for l in range(8):
  for m in range(-l,l+1):
    print(l,m)
    plt.semilogy(abd.t,diff_abs[:, lm(l,m,h.ell_min)],label=f"l,m={l},{m}")
    plt.ylabel("bianchi2")
    plt.xlabel("t")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{save_folder}/l,m={l},{m}.png")
    plt.close()

In [ ]:
save_folder = cce_data.parent/"violations"
save_folder.mkdir(exist_ok=True)
for l in range(8):
  for m in range(-l,l+1):
    print(l,m)
    plt.semilogy(abd.t,diff_rel[:, lm(l,m,h.ell_min)],label=f"l,m={l},{m}")
    plt.ylabel("bianchi2")
    plt.xlabel("t")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{save_folder}/rel_l,m={l},{m}.png")
    plt.close()

In [ ]:
norms = np.linalg.norm(diff, axis=1)

In [ ]:
plt.semilogy(abd.t,norms)

In [ ]:
violations = abd.bondi_violation_norms

In [ ]:
violation_dict = {
  't': abd.t,
  '0': violations[0],
  '1': violations[1],
  '2': violations[2],
  '3': violations[3],
  '4': violations[4],
  '5': violations[5]
}

# with open(cce_data.parent/"violation_dict.pkl",'wb') as f:
#   pickle.dump(violation_dict,f)

In [ ]:

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
plt.semilogy(violation_dict['t'],violation_dict['5'],linestyle='--',label='5',color=colors[5])
plt.semilogy(violation_dict['t'],violation_dict['4'],linestyle='--',label='4',color=colors[4])
plt.semilogy(violation_dict['t'],violation_dict['3'],linestyle='--',label='3',color=colors[3])
plt.semilogy(violation_dict['t'],violation_dict['2'],linestyle='--',label='2',color=colors[2])
plt.semilogy(violation_dict['t'],violation_dict['1'],linestyle='--',label='1',color=colors[1])
plt.semilogy(violation_dict['t'],violation_dict['0'],linestyle='--',label='0',color=colors[0])

plt.xlabel('t')
plt.ylabel("violations")
plt.legend()
plt.tight_layout()
plt.savefig(cce_data.parent/"violations.png")

In [ ]:
violations[2]

In [ ]:
violation_dict = {
  't': abd.t,
  '0': violations[0],
  '1': violations[1],
  '2': violations[2],
  '3': violations[3],
  '4': violations[4],
  '5': violations[5]
}

violation_dict2 = data['Lev5']['0258']

In [ ]:

minT = violation_dict['t'].min()
maxT = violation_dict['t'].max()
print(f"{minT=}, {maxT=}")
mask = (violation_dict2['t'] < maxT) & (violation_dict2['t']  > minT)
# maxT = 200
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
plt.semilogy(violation_dict2['t'][mask],violation_dict2['5'][mask],label='5', color=colors[5])
plt.semilogy(violation_dict2['t'][mask],violation_dict2['4'][mask],label='4', color=colors[4])
plt.semilogy(violation_dict2['t'][mask],violation_dict2['3'][mask],label='3', color=colors[3])
plt.semilogy(violation_dict2['t'][mask],violation_dict2['2'][mask],label='2', color=colors[2])
plt.semilogy(violation_dict2['t'][mask],violation_dict2['1'][mask],label='1', color=colors[1])
plt.semilogy(violation_dict2['t'][mask],violation_dict2['0'][mask],label='0', color=colors[0])

plt.semilogy(violation_dict['t'],violation_dict['5'],linestyle='--',label='2200_5',color=colors[5])
plt.semilogy(violation_dict['t'],violation_dict['4'],linestyle='--',label='2200_4',color=colors[4])
plt.semilogy(violation_dict['t'],violation_dict['3'],linestyle='--',label='2200_3',color=colors[3])
plt.semilogy(violation_dict['t'],violation_dict['2'],linestyle='--',label='2200_2',color=colors[2])
plt.semilogy(violation_dict['t'],violation_dict['1'],linestyle='--',label='2200_1',color=colors[1])
plt.semilogy(violation_dict['t'],violation_dict['0'],linestyle='--',label='2200_0',color=colors[0])

plt.xlabel('t')
plt.ylabel("violations")
plt.legend()

In [ ]:

minT = violation_dict2['t'].min()
maxT = violation_dict2['t'].max()
print(f"{minT=}, {maxT=}")
mask = (violation_dict2['t'] < maxT) & (violation_dict2['t']  > minT)
# maxT = 200
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
plt.semilogy(violation_dict2['t'][mask],violation_dict2['5'][mask],label='5', color=colors[5])
plt.semilogy(violation_dict2['t'][mask],violation_dict2['4'][mask],label='4', color=colors[4])
plt.semilogy(violation_dict2['t'][mask],violation_dict2['3'][mask],label='3', color=colors[3])
plt.semilogy(violation_dict2['t'][mask],violation_dict2['2'][mask],label='2', color=colors[2])
plt.semilogy(violation_dict2['t'][mask],violation_dict2['1'][mask],label='1', color=colors[1])
plt.semilogy(violation_dict2['t'][mask],violation_dict2['0'][mask],label='0', color=colors[0])

plt.xlabel('t')
plt.ylabel("violations")
plt.legend()

In [ ]:
main_folder = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi")

In [ ]:
bondi_violations_dicts = list(main_folder.glob("*Lev*/bondi*"))
bondi_violations_dicts = list(main_folder.glob("Lev5_R0258/bondi*"))

In [ ]:
bondi_violations_dicts.sort()
bondi_violations_dicts

In [ ]:
import pickle

data = {
  "Lev3":{},
  "Lev4":{},
  "Lev5":{},
  "Lev5_old":{}
}
for file in bondi_violations_dicts:
  with file.open('rb') as f:
    lev = str(file).split("/")[-2][3]
    radius = str(file).split("/")[-2][-4:]
    if radius == "_old":
      lev = lev+"_old"
      radius = str(file).split("/")[-2][-8:-4]
    print(lev,radius)
    data["Lev"+lev][radius] = pickle.load(f)


In [ ]:
levs = data.keys()


bondi_violation = '2'
minT = 220
maxT = 7200
# maxT = 200
RADISU = 200

for lev in levs:
  # if lev!='Lev5':
  #   continue
  for radius in data[lev].keys():
    # if int(radius) != RADISU:
    #   continue
    if int(radius) < 150:
      continue
    # print(radius)

    t = data[lev][radius]['t']
    y = data[lev][radius][bondi_violation]

    mask = (t < maxT) & (t > minT)
    if lev == "Lev3":
      # plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-')
      plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-')
    elif lev == 'Lev4':
      # plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='--')
      plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='--')
    elif lev == 'Lev5':
      # plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='--')
      plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='--')
    else:
      # plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-.')
      plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-.')


  plt.xlabel('t')
  # plt.ylabel(bondi_violation)
  plt.title(f"Bondi Violation {bondi_violation}")
  plt.legend()
  plt.savefig(f"./{lev}.png")
  plt.close()

for radius in data['Lev5'].keys():
  for lev in levs:
    # if 'Lev5' not in lev:
    #   continue

    t = data[lev][radius]['t']
    y = data[lev][radius][bondi_violation]

    mask = (t < maxT) & (t > minT)
    if lev == "Lev3":
      # plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-')
      plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-')
    elif lev == 'Lev4':
      # plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='--')
      plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='--')
    elif lev == 'Lev5':
      # plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='--')
      plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='--')
    else:
      # plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-.')
      plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-.')


  plt.xlabel('t')
  # plt.ylabel(bondi_violation)
  plt.title(f"Bondi Violation {bondi_violation}")
  plt.legend()
  plt.tight_layout()
  plt.savefig(f"./{radius}.png")
  plt.close()

In [ ]:
levs = data.keys()


bondi_violation = '2'
minT = 2150
maxT = 7500
RADISU = 900

for lev in levs:
  # if lev!='Lev5':
  #   continue
  for radius in data[lev].keys():
    # if int(radius) != RADISU:
    #   continue
    if int(radius) < 150:
      continue
    # print(radius)

    t = data[lev][radius]['t']
    y = data[lev][radius][bondi_violation]

    mask = (t < maxT) & (t > minT)
    if lev == "Lev3":
      plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-')
      # plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-')
    elif lev == 'Lev4':
      plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='--')
      # plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='--')
    else:
      plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-.')
      # plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-.')


  plt.xlabel('t')
  # plt.ylabel(bondi_violation)
  plt.title(f"Bondi Violation {bondi_violation}")
  plt.legend()
  plt.savefig(f"./{lev}.png")
  plt.close()

for radius in data['Lev5'].keys():
  for lev in levs:
  # if lev!='Lev5':
  #   continue

    t = data[lev][radius]['t']
    y = data[lev][radius][bondi_violation]

    mask = (t < maxT) & (t > minT)
    if lev == "Lev3":
      plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-')
      # plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-')
    elif lev == 'Lev4':
      plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='--')
      # plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='--')
    else:
      plt.plot(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-.')
      # plt.semilogy(t[mask],y[mask],label=f"{lev}_{radius}",linestyle='-.')


  plt.xlabel('t')
  # plt.ylabel(bondi_violation)
  plt.title(f"Bondi Violation {bondi_violation}")
  plt.legend()
  plt.tight_layout()
  plt.savefig(f"./{int(radius)}.png")
  plt.close()

In [ ]:
main_folder = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35/cce_bondi/Lev5_variations")

In [ ]:
bondi_violations_dicts = list(main_folder.glob("*/bondi*"))
bondi_violations_dicts

In [ ]:
bondi_violations_dicts[0].parent.stem

In [ ]:
import pickle

data = {}
for file in bondi_violations_dicts:
  with file.open('rb') as f:
    data[file.parent.stem] = pickle.load(f)

In [ ]:

bondi_violation = '2'
minT = 215
maxT = 7500
RADISU = 900

for key in data.keys():
  if "test" in key:
    continue
  t = data[key]['t']
  y = data[key][bondi_violation]

  mask = (t < maxT) & (t > minT)
  # plt.plot(t[mask],y[mask],label=f"{key}",linestyle='-.')
  plt.semilogy(t[mask],y[mask],label=f"{key}",linestyle='-.')


plt.xlabel('t')
# plt.ylabel(bondi_violation)
plt.title(f"Bondi Violation {bondi_violation}")
plt.tight_layout()
plt.legend()
plt.savefig(f"./variations.png")
plt.close()

# Compute and save bondi violations

In [ ]:
cce_data_folder = Path("/groups/sxs/hchaudha/spec_runs/high_accuracy_L35_variations/Lev5_big_gaussian/GW_data/")

cce_data_paths = list(cce_data_folder.glob(f"**/red_cce.h5"))
for cce_data in cce_data_paths:
  abd = scri.create_abd_from_h5(
        file_name=str(cce_data),
        file_format="spectrecce_v1",
        # ch_mass=1.0,  # Optional; helpful if known
        # t_interpolate=t_worldtube,  # Optional; for some specified values of `t_worldtube`
        # t_0_superrest=3000,
        # padding_time=400
      )

  violations = abd.bondi_violation_norms
  violation_dict = {
  't': abd.t,
  '0': violations[0],
  '1': violations[1],
  '2': violations[2],
  '3': violations[3],
  '4': violations[4],
  '5': violations[5]
  }

  with open(cce_data.parent/"violation_dict.pkl",'wb') as f:
    pickle.dump(violation_dict,f)


  colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
  plt.semilogy(violation_dict['t'],violation_dict['5'],linestyle='--',label='2200_5',color=colors[5])
  plt.semilogy(violation_dict['t'],violation_dict['4'],linestyle='--',label='2200_4',color=colors[4])
  plt.semilogy(violation_dict['t'],violation_dict['3'],linestyle='--',label='2200_3',color=colors[3])
  plt.semilogy(violation_dict['t'],violation_dict['2'],linestyle='--',label='2200_2',color=colors[2])
  plt.semilogy(violation_dict['t'],violation_dict['1'],linestyle='--',label='2200_1',color=colors[1])
  plt.semilogy(violation_dict['t'],violation_dict['0'],linestyle='--',label='2200_0',color=colors[0])

  plt.xlabel('t')
  plt.ylabel("violations")
  plt.legend()
  plt.tight_layout()
  plt.savefig(cce_data.parent/"violations.png")
  plt.close()

In [ ]:
plt.show()